In [1]:
from pathlib import Path

## Load Jupyter Magics
%load_ext sql

# Import ipython-sql Jupyter extension to create SQL cells
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Choose connection
%sql duckdb:///:memory:
#%sql duckdb:///duckdb.db

/datadrive/home/matinnu/.conda/envs/duckdb/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3194: UserWarning: Config option `displaycon` not recognized by `SqlMagic`.  Did you mean `displaylimit`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


'Connected: @:memory:'

## Genome Table

In [5]:
%%sql
WITH src_taxa AS (
    SELECT * FROM read_csv_auto('../data/tables/df_gtdb_meta.csv', header=True)
),

src_genomes AS (
    SELECT * FROM '../data/tables/df_antismash_6.1.1_summary.csv'
),

stg_taxa AS (
    SELECT
        md5(Domain || Phylum || Class || src_taxa.Order || Family || Genus || Organism) AS tax_id,
        genome_id,
    FROM src_taxa
),

stg_genomes AS(
    SELECT 
        src_genomes.genome_id, 
        stg_taxa.tax_id,
        src_genomes.source,
        src_genomes.strain,
        src_genomes.bgcs_count,
        src_genomes.bgcs_on_contig_edge,
    FROM src_genomes, stg_taxa
    WHERE
        src_genomes.genome_id = stg_taxa.genome_id
)

SELECT * FROM stg_genomes

 * duckdb:///:memory:


,genome_id,tax_id,source,strain,bgcs_count,bgcs_on_contig_edge
0,P5-2B-15,654beafd7c58d4fd0e8dee18a4150eab,custom,P5-2B-15,38.0,1.0
1,P5-2B-6,e8521a3a71f5e9e1abd02ce4746fd755,custom,P5-2B-6,27.0,0.0
2,P5-A2,f662b184b0b0b557d18cf9726e054006,custom,P5-A2,26.0,0.0
3,P5-A3,9699895deb16dc9cf79099ae5cb4e1a0,custom,P5-A3,28.0,0.0
4,P5-A5,9699895deb16dc9cf79099ae5cb4e1a0,custom,P5-A5,27.0,5.0
5,P5-A9,f662b184b0b0b557d18cf9726e054006,custom,P5-A9,26.0,0.0
6,P8-2B-10,9228b746e166896f3f49600d0074c606,custom,P8-2B-10,33.0,0.0
7,P8-2B-3,3101606c4ea30cc83ae925d0eb0d8566,custom,P8-2B-3,22.0,0.0
8,P8-A1,82c47f05c84fe00ad429535c123a02f3,custom,P8-A1,34.0,0.0
9,P8-A2,3f2bd94e2ce499004bdd2063974c443d,custom,P8-A2,39.0,0.0


## Taxa table

In [9]:
%%sql
WITH src_taxa AS (
    SELECT * FROM read_csv_auto('../data/tables/df_gtdb_meta.csv', header=True)
),

stg_taxa AS (
    SELECT
        md5(Domain || Phylum || Class || src_taxa.Order || Family || Genus || Organism) AS tax_id,
        genome_id,
        Domain AS domain,
        Phylum AS phylum,
        Class AS class,
        src_taxa.Order AS taxonomic_order,
        Family AS family,
        Genus AS genus,
        Organism AS species,
    FROM src_taxa
),

src_genomes AS (
    SELECT * FROM "../data/df_antismash_6.1.1_summary.csv"
),

stg_genomes AS(
    SELECT 
        src_genomes.genome_id, 
        src_genomes.source,
        src_genomes.strain,
        src_genomes.bgcs_count,
        src_genomes.bgcs_on_contig_edge ,
        stg_taxa.tax_id
    FROM src_genomes, stg_taxa
    WHERE
        src_genomes.genome_id = stg_taxa.genome_id
)

SELECT DISTINCT tax_id, domain, phylum, class, taxonomic_order, family, genus, species FROM stg_taxa;

 * duckdb:///:memory:


,tax_id,domain,phylum,class,taxonomic_order,family,genus,species
0,654beafd7c58d4fd0e8dee18a4150eab,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces tubercidicus
1,e8521a3a71f5e9e1abd02ce4746fd755,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces halstedii
2,f662b184b0b0b557d18cf9726e054006,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces sp002711365
3,9699895deb16dc9cf79099ae5cb4e1a0,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__
4,9228b746e166896f3f49600d0074c606,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces niveus
5,3101606c4ea30cc83ae925d0eb0d8566,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces albidoflavus
6,82c47f05c84fe00ad429535c123a02f3,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces niveus_A
7,3f2bd94e2ce499004bdd2063974c443d,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces mirabilis
8,b7c1b7cd9c319cdb2d0761a209af5354,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces sp003846175
9,c25b8f5cc3468b556af59c894265a68d,d__Bacteria,p__Actinobacteriota,c__Actinomycetia,o__Streptomycetales,f__Streptomycetaceae,g__Streptomyces,s__Streptomyces papulosus


## Seqfu table

In [15]:
%%sql
WITH src_seqfu AS (
    SELECT * FROM read_csv_auto('../data/tables/df_seqfu_stats.csv', header=True)
),

stg_seqfu AS(
    SELECT
        genome_id,
        src_seqfu.Min AS s_min,
        src_seqfu.N25 AS s_n25,
        src_seqfu.N50 AS s_n50,
        src_seqfu.AuN AS s_aun,
        src_seqfu.gc AS s_gc,
        src_seqfu.Max AS s_max,
        src_seqfu.N75 AS s_n75,
        src_seqfu.Count as s_count,
        src_seqfu.Total as s_total,
        src_seqfu.N90 AS s_n90,
        src_seqfu.Avg AS s_avg
    FROM src_seqfu
)

SELECT * FROM stg_seqfu

 * duckdb:///:memory:


,genome_id,s_min,s_n25,s_n50,s_aun,s_gc,s_max,s_n75,s_count,s_total,s_n90,s_avg
0,P5-2B-15,57728,9001612,9001612,8.944620e+06,0.705358,9001612,9001612,2,9059340,9001612,4.529670e+06
1,P5-2B-6,33363,7332070,7332070,7.299009e+06,0.719287,7332070,7332070,2,7365433,7332070,3.682716e+06
2,P5-A2,42226,7674445,7674445,7.211944e+06,0.711742,7674445,7674445,3,8199128,7674445,2.733043e+06
3,P5-A3,8424100,8424100,8424100,8.424100e+06,0.719898,8424100,8424100,1,8424100,8424100,8.424100e+06
4,P5-A5,7969,8008365,8008365,7.376991e+06,0.698337,8008365,8008365,9,8710355,8008365,9.678172e+05
5,P5-A9,21179,7671826,7671826,7.228330e+06,0.711728,7671826,7671826,3,8174694,7671826,2.724898e+06
6,P8-2B-10,18644,8582327,8582327,8.563764e+06,0.706960,8582327,8582327,2,8600971,8582327,4.300486e+06
7,P8-2B-3,7099240,7099240,7099240,7.099240e+06,0.733603,7099240,7099240,1,7099240,7099240,7.099240e+06
8,P8-A1,3682,8773074,8773074,8.603543e+06,0.703441,8773074,8773074,3,8949412,8773074,2.983137e+06
9,P8-A2,93143,11475542,11475542,1.001415e+07,0.700637,11475542,11475542,6,13234767,554092,2.205794e+06


## CheckM Table

In [24]:
%%sql
WITH src_checkm AS (
    SELECT * FROM read_csv_auto('../data/tables/df_checkm_stats.csv', header=True)
),

stg_checkm AS (
    SELECT
        genome_id,
        Completeness AS completeness,
        Contamination AS contamination,
        "# ambiguous bases" AS num_ambiguous_bases,
        "Coding density" AS coding_density,
        "# predicted genes" AS num_predicted_genes       
    FROM src_checkm
)

SELECT * FROM stg_checkm

 * duckdb:///:memory:


,genome_id,completeness,contamination,num_ambiguous_bases,coding_density,num_predicted_genes
0,P9-2B-2,0.000000,0.000000,0,0.858175,7766
1,P5-A9,0.000000,0.000000,0,0.876789,7362
2,P5-A5,0.000000,0.000000,0,0.854666,7723
3,P5-2B-15,0.000000,0.000000,0,0.856201,7888
4,P8-A1,0.000000,0.000000,0,0.876049,7949
5,P8-A7,0.000000,0.000000,0,0.853119,7619
6,P8-A6,0.000000,0.000000,0,0.870078,8750
7,P8-A8,0.000000,0.000000,0,0.864623,9506
8,P5-A2,0.000000,0.000000,0,0.875863,7391
9,P8-A4-2,0.000000,0.000000,0,0.876605,7244


In [ ]:
"../data/bigscape/for_cytoscape_antismash_6.1.1/"

## BiG-SCAPE table

In [47]:
%%sql
WITH src_bigscape_cluster AS (
    SELECT * FROM read_csv_auto('../data/bigscape/for_cytoscape_antismash_6.1.1/*_df_clusters_0.30.csv', header=True)
),

stg_bigscape_cluster AS (
    SELECT
        bgc_id,
        genome_id,
        product,
        bigscape_class,
        accn_id,
        "gcf_0.30" AS gcf,
        "Clan Number" AS clan_number,
        "fam_id_0.30" AS fam_id,
        "fam_type_0.30" AS fam_type,
        "fam_known_compounds_0.30" AS fam_known_compounds
    FROM src_bigscape_cluster
)

SELECT * FROM stg_bigscape_cluster

 * duckdb:///:memory:


,bgc_id,genome_id,product,bigscape_class,accn_id,gcf,clan_number,fam_id,fam_type,fam_known_compounds
0,P5-2B-15_2.region006,P5-2B-15,T3PKS.lanthipeptide-class-iii,Others,P5-2B-15_2,1917.0,NaN,56.0,unknown_family,u_Others_56
1,P5-2B-15_2.region010,P5-2B-15,NRPS-like.NRPS.T1PKS,PKS-NRP_Hybrids,P5-2B-15_2,1921.0,908.0,348.0,unknown_family,u_PKS-NRP_Hybrids_348
2,P5-2B-15_2.region013,P5-2B-15,siderophore,Others,P5-2B-15_2,2443.0,NaN,25.0,unknown_family,u_Others_25
3,P5-2B-15_2.region026,P5-2B-15,NRPS.RiPP-like,Others,P5-2B-15_2,1937.0,NaN,218.0,unknown_family,u_Others_218
4,P5-2B-15_2.region001,P5-2B-15,NRPS,NRPS,P5-2B-15_2,1912.0,308.0,173.0,unknown_family,u_NRPS_173
...,...,...,...,...,...,...,...,...,...,...
754,P9-A2_1.region003,P9-A2,NRPS-like.T1PKS,PKS-NRP_Hybrids,P9-A2_1,2496.0,908.0,353.0,unknown_family,u_PKS-NRP_Hybrids_353
755,P9-A2_1.region019,P9-A2,T1PKS.phenazine,Others,P9-A2_1,2512.0,NaN,244.0,unknown_family,u_Others_244
756,P9-A2_1.region018,P9-A2,NRPS,NRPS,P9-A2_1,2511.0,308.0,147.0,unknown_family,u_NRPS_147
757,P9-A2_1.region009,P9-A2,redox-cofactor,Others,P9-A2_1,2502.0,NaN,270.0,unknown_family,u_Others_270


## BiG-SCAPE network

In [57]:
%%sql
WITH src_bigscape_network AS (
    SELECT * FROM read_csv_auto('../data/bigscape/for_cytoscape_antismash_6.1.1/*_df_network_0.30.csv', header=True)
),

stg_bigscape_network AS (
    SELECT
        bigscape_edge_id,
        "Clustername 1" AS clustername_1,
        "Clustername 2" AS clustername_2,
        "Raw distance" AS raw_distance,
        "Squared similarity" AS squared_similarity,
        "Jaccard index" AS jaccard_index,
        "DSS index" AS dss_index,
        "Adjacency index" AS adjacency_index,
        "raw DSS non-anchor" AS raw_DSS_non_anchor,
        "raw DSS anchor" AS raw_DSS_anchor,
        "Non-anchor domains" AS non_anchor_domains,
        "Anchor domains" AS anchor_domains,
        "Combined group" AS combined_group,
        "Shared group" AS shared_group
    FROM src_bigscape_network
)

SELECT * FROM stg_bigscape_network

 * duckdb:///:memory:


,bigscape_edge_id,clustername_1,clustername_2,raw_distance,squared_similarity,jaccard_index,dss_index,adjacency_index,raw_DSS_non_anchor,raw_DSS_anchor,non_anchor_domains,anchor_domains,combined_group,shared_group
0,0,BGC0000324.1,P5-2B-15_2.region032,0.170036,0.688840,1.000000,0.829964,1.000000,0.139641,0.192832,21,7,NRPS - NRPS,NRPS
1,1,P8-A6_3.region026,P8-A8_7.region001,0.025490,0.949670,0.958333,0.974510,0.925926,0.029412,0.000000,26,1,NAPAA - NRPS.NAPAA,None
2,2,P8-A8_7.region002,P9-A8_8.region001,0.058587,0.886258,0.897436,0.941413,0.760870,0.018963,0.086104,50,18,NRPS - NRPS.NAPAA,None
3,3,P8-A4-2_3.region022,P9-A4_3.region004,0.075995,0.853785,0.945946,0.924005,0.822222,0.039905,0.108616,47,13,NRPS - NRPS,NRPS
4,4,P8-A6_3.region021,P9-A8_8.region006,0.034787,0.931636,0.942857,0.965213,0.844444,0.050632,0.013000,44,8,NRPS - NRPS,NRPS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1539,3355,P8-A7_4.region002,P8-A7_4.region002,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1,1,None,None
1540,3358,P5-A3_2.region008,P5-A3_2.region008,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1,1,None,None
1541,3365,P9-A3_2.region008,P9-A3_2.region008,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1,1,None,None
1542,3366,P5-2B-6_2.region015,P5-2B-6_2.region015,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1,1,None,None


## MIBIG Hits

In [51]:
%%sql
WITH src_bigscape_mibig_hits AS (
    SELECT * FROM read_csv_auto('../data/bigscape/for_cytoscape_antismash_6.1.1/*_df_known_0.30.csv', header=True)
),

stg_bigscape_mibig_hits AS (
    SELECT
        bgc_id,
        product,
        bigscape_class,
        biosyn_class,
        compounds,
        chem_acts,
        accession,
        completeness,
        organism_name,
        ncbi_tax_id,
        publications,
        evidence,
        "gcf_0.30" AS gcf,
        "Clan Number" AS clan_number,
        "fam_id_0.30" AS fam_id,
        "fam_type_0.30" AS fam_type,
        "fam_known_compounds_0.30" AS fam_known_compounds
    FROM src_bigscape_mibig_hits
)

SELECT * FROM stg_bigscape_mibig_hits

 * duckdb:///:memory:


,bgc_id,product,bigscape_class,biosyn_class,compounds,chem_acts,accession,completeness,organism_name,ncbi_tax_id,publications,evidence,gcf,clan_number,fam_id,fam_type,fam_known_compounds
0,BGC0000029.1,T1PKS,PKSI,Polyketide,BE-14106,Cytotoxic,FJ872523.1,complete,Streptomyces sp. DSM 21069,639005.0,pubmed:19875084,Knock-out studies;Enzymatic assays,1262.0,NaN,32.0,known_family,heronamide A;heronamide B;heronamide C;heronam...
1,BGC0000034.1,NRPS-like.T1PKS,PKS-NRP_Hybrids,Polyketide,candicidin,None,AY310323.2,Unknown,Streptomyces sp. FR-008,206662.0,pubmed:14652074,None,2136.0,908.0,68.0,known_family,levorin A3;C06690;FR-008-III;candicidin A;UNII...
2,BGC0000061.1,NRPS-like.T1PKS,PKS-NRP_Hybrids,Polyketide,levorin A3;C06690;FR-008-III;candicidin A;UNII...,None,AY310323.2,Unknown,Streptomyces sp. FR-008,206662.0,pubmed:14652074,None,2136.0,908.0,68.0,known_family,levorin A3;C06690;FR-008-III;candicidin A;UNII...
3,BGC0000097.1,T1PKS,PKSI,Polyketide,ML-449,Antibacterial;Antifungal,FJ872525.1,complete,Streptomyces sp. MP39-85,639006.0,pubmed:19854930,Knock-out studies,1262.0,NaN,32.0,known_family,heronamide A;heronamide B;heronamide C;heronam...
4,BGC0000115.1,T1PKS,PKSI,Polyketide;Saccharide,nystatin A1,Antifungal,AF263912.1,complete,Streptomyces noursei ATCC 11455,316284.0,pubmed:10873841,Knock-out studies,110.0,NaN,31.0,known_family,nystatin A1;nystatin-like Pseudonocardia polye...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,BGC0001920.1,lassopeptide,RiPPs,RiPP,siamycin I,None,LC381634.1,Unknown,Streptomyces sp.,1931.0,pubmed:29974638,None,1887.0,2045.0,12.0,known_family,MS-271;aborycin;MS-271;siamycin I
68,BGC0001921.1,lassopeptide,RiPPs,Other,MS-271,None,LC381634.1,Unknown,Streptomyces sp.,1931.0,pubmed:29974638,None,1887.0,2045.0,12.0,known_family,MS-271;aborycin;MS-271;siamycin I
69,BGC0001937.1,nucleoside,Others,Other,tubercidin,None,MG706975.1,Unknown,Streptomyces tubercidicus,47759.0,pubmed:30153835,None,1810.0,NaN,102.0,known_family,tubercidin
70,BGC0002014.1,ladderane.NRPS.arylpolyene,Others,NRP;Polyketide,pepticinnamin E,None,NZ_LJCU01000236.1,Unknown,Actinobacteria bacterium OK006,1592326.0,pubmed:30694017,None,2190.0,NaN,110.0,known_family,pepticinnamin E


## BiG-FAM Query

In [61]:
%%sql
WITH src_bigfam_hits AS (
    SELECT * FROM read_csv_auto('../data/bigslice/query_as_6.1.1/gcf_summary.csv', header=True)
),

stg_bigfam_hits AS (
    SELECT 
        gcf_id AS bigfam_id,
        core_member,
        putative_member,
        core_member_mibig,
        putative_member_mibig,
        core_member_mibig_count,
        core_member_mibig_bool,
        putative_member_mibig_count,
        putative_member_mibig_bool,
        "link to BiG-FAM" AS bigfam_link
    FROM src_bigfam_hits
)

SELECT * FROM stg_bigfam_hits

 * duckdb:///:memory:


,bigfam_id,core_member,putative_member,core_member_mibig,putative_member_mibig,core_member_mibig_count,core_member_mibig_bool,putative_member_mibig_count,putative_member_mibig_bool,bigfam_link
0,200946,131525,4659,"['BGC0001286.1', 'BGC0001864.1', 'BGC0002005.1...","['BGC0000736.1', 'BGC0000883.1', 'BGC0000765.1...",124,True,29,True,https://bigfam.bioinformatics.nl/run/6/gcf/200946
1,203037,8270,2009,[],[],0,False,0,False,https://bigfam.bioinformatics.nl/run/6/gcf/203037
2,210179,9091,1159,"['BGC0001201.1', 'BGC0001181.1', 'BGC0001911.1...","['BGC0001910.1', 'BGC0001322.1', 'BGC0001324.1']",16,True,3,True,https://bigfam.bioinformatics.nl/run/6/gcf/210179
3,213140,1871,1165,"['BGC0001928.1', 'BGC0001893.1', 'BGC0001605.1...","['BGC0001882.1', 'BGC0001985.1', 'BGC0000887.1...",9,True,9,True,https://bigfam.bioinformatics.nl/run/6/gcf/213140
4,208355,6440,3186,"['BGC0000595.1', 'BGC0000596.1', 'BGC0000933.1...","['BGC0001137.1', 'BGC0001295.1']",6,True,2,True,https://bigfam.bioinformatics.nl/run/6/gcf/208355
...,...,...,...,...,...,...,...,...,...,...
180,205397,80,0,[],[],0,False,0,False,https://bigfam.bioinformatics.nl/run/6/gcf/205397
181,209843,1,0,[],[],0,False,0,False,https://bigfam.bioinformatics.nl/run/6/gcf/209843
182,218426,8,0,[],[],0,False,0,False,https://bigfam.bioinformatics.nl/run/6/gcf/218426
183,209562,10,1,[],[],0,False,0,False,https://bigfam.bioinformatics.nl/run/6/gcf/209562


## BiG-FAM Query Network

In [63]:
%%sql
WITH src_bigfam_network as (
    SELECT * FROM read_csv_auto('../data/bigslice/query_as_6.1.1/query_network.csv', header=True)
),

stg_bigfam_network as (
    SELECT
        gcf_id AS bigfam_id,
        bgc_id,
        membership_value,
        rank
    FROM src_bigfam_network
)

SELECT * FROM stg_bigfam_network

 * duckdb:///:memory:


,bigfam_id,bgc_id,membership_value,rank
0,200946,P5-A2_2.region011,436,0
1,203037,P5-A2_2.region011,552,1
2,210179,P5-A2_2.region011,553,2
3,213140,P5-A2_2.region011,559,3
4,208355,P5-A2_2.region011,569,4
...,...,...,...,...
1296,209843,P8-2B-10_1.region030,741,0
1297,218426,P8-2B-3_2.region013,475,0
1298,209562,P5-2B-6_2.region008,693,0
1299,204252,P9-2B-3_1.region028,559,0
